# Read and prepare Dataset

In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
pn.extension()
import hvplot.pandas

In [2]:
#read dataset
data = pd.read_csv('D:/1TU/Wirtschaftsinformatik/Bachelorarbeit/Dashboard/learning_analytics_dashboard/example_data_set.csv', low_memory=False)

In [3]:
#add failed columns
data['exercise 1 failed'] = np.where(data['exercise 1 passed'] == 0, 1, 0)
data['exercise 2 failed'] = np.where(data['exercise 2 passed'] == 0, 1, 0)
data['exercise 3 failed'] = np.where(data['exercise 3 passed'] == 0, 1, 0)
data['exercise 4 failed'] = np.where(data['exercise 4 passed'] == 0, 1, 0)
data['exercise 5 failed'] = np.where(data['exercise 5 passed'] == 0, 1, 0)
data['exercise 6 failed'] = np.where(data['exercise 6 passed'] == 0, 1, 0)

#add subject column
data['subject'] = ""
index = 0
subject_list = ['Deutsch', 'Englisch', 'Mathematik', 'Biologie', 'Musik', 'Kunst', 'Sport']
subject_index = 0

while(index < len(data)):
    #data.at['subject', index] = subject_list[subject_index]
    data.loc[index, 'subject'] = subject_list[subject_index]
    index += 1
    if(index > subject_index*10):
        subject_index += 1
    if(subject_index >= len(subject_list)):
        subject_index = 0

#add class column
data['class'] = ""
index = 0
class_list = ['Klasse 1a', 'Klasse 1b', 'Klasse 3b', 'Klasse 4c', 'Klasse 4e', 'Klasse 5a', 'Klasse 6a']
class_index = 0

while(index < len(data)):
    data.loc[index, 'class'] = class_list[class_index]
    index += 1
    if(index > class_index*30):
        class_index += 1
        
data

,name,exercise 1 date,exercise 1 time,exercise 1 score,exercise 1 passed,exercise 1 duration,exercise 1 number of tries,exercise 2 date,exercise 2 time,exercise 2 score,...,exercise 6 duration,exercise 6 number of tries,exercise 1 failed,exercise 2 failed,exercise 3 failed,exercise 4 failed,exercise 5 failed,exercise 6 failed,subject,class
0,Haviva Welch,"Aug 11, 2023",1:35 AM,21,0,0:18:17,3,"Apr 24, 2023",1:03 PM,48,...,7:22:17,6,1,1,1,0,0,1,Deutsch,Klasse 1a
1,Kyle Grimes,"Aug 13, 2023",2:54 AM,42,1,1:29:11,3,"Oct 18, 2023",10:59 AM,49,...,9:43:29,56,0,0,1,1,1,0,Englisch,Klasse 1b
2,Raphael Hickman,"Aug 21, 2023",2:15 PM,19,1,0:22:55,2,"Aug 12, 2024",7:33 PM,23,...,17:04:45,73,0,1,1,1,1,1,Englisch,Klasse 1b
3,Illana Acevedo,"Aug 12, 2023",8:35 PM,90,1,0:47:26,0,"Oct 22, 2022",8:40 AM,65,...,1:47:21,75,0,1,1,1,0,0,Englisch,Klasse 1b
4,Pearl Silva,"Aug 7, 2023",8:13 AM,85,1,0:13:14,1,"Jun 4, 2023",12:27 AM,69,...,9:07:14,25,0,0,0,1,1,1,Englisch,Klasse 1b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Erin Rollins,"Aug 19, 2023",6:52 AM,54,1,1:09:46,1,"Oct 7, 2023",7:26 AM,95,...,10:49:03,8,0,0,0,0,0,1,Deutsch,Klasse 5a
146,Lane Browning,"Aug 24, 2023",11:31 PM,96,1,1:17:02,2,"Jun 7, 2023",11:27 PM,31,...,16:27:25,84,0,0,1,0,1,1,Englisch,Klasse 5a
147,Gail Navarro,"Aug 12, 2023",11:44 AM,47,0,0:10:47,2,"Nov 4, 2022",3:37 AM,30,...,12:00:30,14,1,1,1,0,1,0,Mathematik,Klasse 5a
148,Quentin Whitfield,"Aug 5, 2023",11:28 PM,48,1,0:05:29,2,"Feb 13, 2024",4:40 AM,95,...,6:58:31,57,0,1,0,1,1,1,Biologie,Klasse 5a


***
# Functionality: Create buttons and menus

## Button to choose exercise

In [4]:
#create col name lists for buttons
exercise_1_list = [col for col in data.columns if 'exercise 1' in col]
exercise_2_list = [col for col in data.columns if 'exercise 2' in col]
exercise_3_list = [col for col in data.columns if 'exercise 3' in col]
exercise_4_list = [col for col in data.columns if 'exercise 4' in col]
exercise_5_list = [col for col in data.columns if 'exercise 5' in col]
exercise_6_list = [col for col in data.columns if 'exercise 6' in col]

#exercise buttons
exercise_buttons = pn.Row(
    *(pn.Column(
        pn.widgets.RadioButtonGroup( 
            button_type = 'success',
            #button_style = 'solid', 
            options = ['Allgemein', 'Aufgabe 1', 'Aufgabe 2', 'Aufgabe 3', 'Aufgabe 4', 'Aufgabe 5','Aufgabe 6'], 
            value = 'Allgemein',
            orientation = 'vertical'
        
        #for exercise_buttons in pn.widgets.Button.param.button_type.objects 
))))
exercise_buttons

Row
    [0] RadioButtonGroup(button_type='success', options=['Allgemein', ...], orientation='vertical', value='Allgemein')

## Button to choose subject

In [5]:
subject_buttons = pn.Row(
    *(pn.Column(
        pn.widgets.RadioButtonGroup( 
            button_type = 'success',
            #button_style = 'solid', 
            options = subject_list, 
            value = subject_list[0],
            orientation = 'horizontal'
        
        #for exercise_buttons in pn.widgets.Button.param.button_type.objects 
))))
subject_buttons

Row
    [0] RadioButtonGroup(button_type='success', options=['Deutsch', 'Englisch', ...], value='Deutsch')

## Menu to choose class and student

In [6]:
name_list = data['name'].values.tolist()
name_list.sort()
menu_items = [*class_list, None, *name_list]
#menu_items

#class_student_menu = pn.widgets.MenuButton(name='Wähle Klasse/Schüler*in', items=menu_items, button_type='primary')
#pn.Column(class_student_menu, height=200)

#class_student_selection = pn.widgets.Select(name='Wähle Klasse/Schüler*in', options = menu_items)
class_student_selection = pn.widgets.Select(name='Wähle Klasse/Schüler*in aus', groups = {'Klasse': class_list, 'Schüler*in': name_list})
class_student_selection
    

Select(groups={'Klasse': ['Klasse 1a', ...}, name='Wähle Klasse/Schüler*in a..., value='Klasse 1a')

## Menu to choose date range

# Create Dashboard

In [7]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Mein Dashboard', 
    sidebar=[pn.pane.Markdown("# Auswahl"),  
             class_student_selection,
             exercise_buttons, 
             pn.pane.Markdown("## Einstellungen")],
    #main=[pn.Row(pn.Column(subject_buttons(width=700), margin=(0,25)))],
    main=[pn.Column(subject_buttons)],
    accent = '#88d8b0'
)
# template.show()
template.servable();

#command: panel serve learning_analytics_dashboard.ipynb in terminal then click on link